In [1]:
import numpy as np

# Variance Reduction Value Target

$$ L = \frac{1}{n} \sum_{i = 0}^{n} (V(s) - A(s)_{\text{norm}} + \hat{V(s)})^2 $$

$$ L = \frac{1}{n} \sum_{i = 0}^{n} (\frac{A - A_{\text{mean}}}{A_{\text{std}}})^2 = 1 $$

https://en.wikipedia.org/wiki/Variance

$$ Var[X] = E[X^2] - E[X]^2 $$

$$ E[X^2] = Var[X] + E[X]^2 $$

In [27]:
X = np.random.normal(scale=1, size=1000000)

In [29]:
np.mean(X**2)

0.9996216398395751